## Imports 

In [1]:
#Versión para tweepy V2
import datetime
from datetime import date
import tweepy
import csv
import os
import pandas as pd
import json
from subprocess import check_output
import time
from pathlib import Path
import math
import pymongo
from bson import ObjectId

## Constant and Variables

In [2]:
# CONSTANTS
PETITIONS_PER_TIME_LAPSE = 300
TIME_LAPSE = 15*60
TIMER = TIME_LAPSE/PETITIONS_PER_TIME_LAPSE
TIMER_LOG = 2*60
listaResultados = []

PETITIONS_LIMIT = 100

#VARIABLES
iniTimerLog = 0
finTimerLog = 0
tiempo = TIMER_LOG
entrada = ""
next_token = None
entradaizq = '' 
entradader = ''
hashtags = ''
total = 0
startDate = ''
endDate = ''
inicio = 0
fin = 0

hashtag_ini_log = ''
start_log_timer = ''
hashtag_last_log = ''
tweet_count = 0

url = 'curl "https://api.twitter.com/2/tweets/search/all?'
today = datetime.date.today()
d1 = today.strftime("%d-%m-%Y")



## Support functions

In [3]:
def DayMonthYearToPetition(date):
    splited = ''
    if ('/' in date):
        splited = date.split('/')
    elif ('-' in date):
        splited = date.split('-')

    return (splited[2] + '-' + splited[1] + '-' + splited[0])

def PetitionToDayMonthYear(petition):
    splited = ''
    time = ''
    splited = petition.split('T')
    time = splited[1]
    splited = splited[0].split('-')
    return (splited[2] + '-' + splited[1] + '-' + splited[0] + 'T' + time)

def make_objid(text):
    """Makes an ObjectId of 4 bytes
    
    Keyword arguments:
    text -- string to be converted into Object ID
    """
    text = str(text)
    if not text.strip():
        return None
    try:
        return ObjectId(text.rjust(24,"0"))
    except Exception as ex:
        print(len(text))
        print(text, ex)
        return None


user_dictionary = {
    "_id"               : None,
    "description"       : '',
    "created_at"        : '',
    "verified"          : '',
    "followers_count"   : 0,
    "following_count"   : 0
}


tweet_dictionary = {
    "_id"               : None,
    "author_id"         : '',
    "created_at"        : '',
    "text"              : '',
    "hashtag"           : '',
    "referenced_tweets" : [],
    "retweet_count"     : 0,
    "reply_count"       : 0,
    "reply_count"       : 0,
    "like_count"        : 0
}

# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# next_token: token de la page (inicialmente string vac'ia) que quieres buscar
# return tupla de n'umero de respuestas y next_token
def PetitionsLessEqual100(hashtag, start, end, number, bearer, next_token):

    startDate = DayMonthYearToPetition(start)
    endDate = DayMonthYearToPetition(end)
    tweet_list = []
    user_list = []
    total = (0, '', [], [])
    dt_string = ''
    tweet_dictionary_new = {}
    user_dictionary_new = {}

    # Insertados los siguientes campos:
    #   -start_time: límite inferior en búsqueda de fechas
    #   -end_time: límete superior en búsqueda de fechas
    #   -tweet.fields=author_id: id del autor del tweet
    #   -tweet.fields=referenced_tweets: puede ser quote, retweet o replied_to junto al id al que responde, agrega además los tweets relativos al tweet al que responde
    #   -tweet.fields=created_at: cu'ando se cre'o el fichero
    #   -tweet.fields=context_annotations:anotaciones de contexto (no salen muchas)
    #   -tweet.fields=lang: Abreviatura del idioma en el que se escribe
    #   -tweet.fields=entities: referencias, hashtags usados por el usuario
    #   -tweet.fields=in_reply_to_user_id: 
    #   -    in_reply_to_user_id
    entradaizq = url + "query=%23" + hashtag + "&start_time=" +  startDate + "T00%3A00%3A00Z" + "&end_time=" + endDate + 'T11%3A59%3A59Z&max_results=' + str(number) + '&expansions=author_id,referenced_tweets.id&tweet.fields=created_at,conversation_id,referenced_tweets,public_metrics&user.fields=created_at,entities,description,verified,public_metrics'
    entradader = '"' + ' -H "Authorization: Bearer '+ bearer + '"'

    if (next_token != ''):
        entrada = entradaizq + '&next_token=' + next_token + entradader
    else:
        entrada = entradaizq + entradader

    # primera petici'on del hashtag
    json_obj = json.loads(check_output(entrada, shell=True).decode())

    # inicio de timer
    # Comprobar que hay respuestas para dicho hashtag
    if ('meta' in json_obj) and (int(json_obj['meta']['result_count']) > 0):
        for tweet in json_obj['data']:
            tweet_dictionary_new = tweet_dictionary.copy()
            tweet_dictionary_new['referenced_tweets'] = []
            user_dictionary_new = user_dictionary.copy()
            

            tweet_dictionary_new['author_id'] = tweet['author_id']
            tweet_dictionary_new['text'] = tweet['text']
            tweet_dictionary_new['_id'] = make_objid(str(tweet['id']))
            tweet_dictionary_new['created_at'] = tweet['created_at']

            tweet_dictionary_new['retweet_count'] = tweet['public_metrics']['retweet_count']
            tweet_dictionary_new['reply_count'] = tweet['public_metrics']['reply_count']
            tweet_dictionary_new['like_count'] = tweet['public_metrics']['like_count']
            tweet_dictionary_new['quote_count'] = tweet['public_metrics']['quote_count']

            
            if ('referenced_tweets' in tweet):
                tweet_dictionary_new['referenced_tweets'] = tweet['referenced_tweets']
                for rt in tweet_dictionary_new['referenced_tweets']:
                    for tweetexpanse in json_obj['includes']['tweets']:
                        if (rt['id'] == tweetexpanse['id']):
                            rt['author_id'] = tweetexpanse['author_id']
                            break

            unknown = True
            for user in user_list:
                if (str(user['_id']) == tweet_dictionary_new['author_id']):
                    unknown = False
                    break

            if (unknown == True):
                user_dictionary_new['_id'] = make_objid(str(tweet['author_id']))
                if ('includes' in json_obj):
                    for user in json_obj['includes']['users']:
                        if (str(user['id']) == tweet['author_id']):
                            user_dictionary_new['description'] = user['description']
                            user_dictionary_new['created_at'] = user['created_at']
                            user_dictionary_new['verified'] = user['verified']
                            user_dictionary_new['followers_count'] = int(user['public_metrics']['followers_count'])
                            user_dictionary_new['following_count'] = int(user['public_metrics']['following_count'])
                            break
                #aqui otro manejo de errores
                else:
                    print ('otro error')
                user_list.append(user_dictionary_new)
            tweet_list.append(tweet_dictionary_new)

        if ('next_token' in json_obj['meta']):
            total = (int(json_obj['meta']['result_count']), json_obj['meta']['next_token'], tweet_list, user_list)

        else:
            total = (int(json_obj['meta']['result_count']), '', tweet_list, user_list)

        print("N'umero de peticiones enviadas: " + str(number))
        print("N'umero de peticiones recibidas: " + str(json_obj['meta']['result_count']))
        return total

    #si no tiene meta entonces es un error y hay que manejarlo
    elif ('meta' in  json_obj) and (int(json_obj['meta']['result_count']) == 0):
        # datetime object containing current date and time
        now = datetime.datetime.now()
        # dd/mm/YY H:M:S
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        errorlog.write("Error, no hay tweets para el Hashtag: " + str(hashtag) + " " + dt_string + '\n')
        print("No hay tweets para el Hashtag: " + str(hashtag))
    else:
        strerror = ''
        print("Error en la petición: ")
        strerror = strerror + "Error en la petición: "
        # si el error es por cuestión de peticiones esperar para hacer la siguiente peticion
        if ('title' in json_obj):
            print("  Título del error: " + json_obj['title'])
            strerror = strerror + "  Título del error: " + str(json_obj['title'])
            if ('status' in json_obj):
                strerror = strerror + " Código de error: " + str(json_obj['status'])
                print("  Código de error: " + json_obj['status'])
            if ('detail' in json_obj):
                strerror = strerror + "  Descripción del error: " + str(json_obj['detail'])
                print("  Descripción del error: " + json_obj['detail'])

            if ('Invalid Request' == json_obj['title']):
                strerror = strerror + " Error: " + str(entrada.split(' -H')[0])
                print(entrada.split(' -H')[0])
        errorlog.write(strerror + " " + dt_string + '\n')
    return total




# Hashtag: nombre del hashtag que se va a buscar
# start: fecha de inicio
# end: fecha de respuesta
# number: n'umero de tweets que queremos obtener
# bearer: bearer id
# return devuelve dos diccionarios (users-tweet, tweet-tweet_items)
def TweetList(hashtag, start, end, number, bearer):
    global tiempo
    global tweet_count
    global hashtag_ini_log
    global hashtag_last_log
    global start_log_timer

    #aqui deberia de hacer dos variables de incio y fin de log para saber con cual hashtag empiezo y termino y con cuantos valores totales de cada uno.
    user_list = []
    tweet_list = []
    total = 0
    aux = 0
    startTime = 0
    endTime = 0
    response = (0, '', [], [])

    print("Inicio Hashtag: " + hashtag + "\\\\\\\\\\\\ \n")
    if (number < PETITIONS_LIMIT):
        startTime = time.time()
        if (number < 10):
            response =  PetitionsLessEqual100(hashtag, start, end, 10, bearer, None)
        else:
            response =  PetitionsLessEqual100(hashtag, start, end, number, bearer, None)
        # insercion de usuarios nuevos y de tweets nuevos de usuarios ya insertados
        print("lista de usuario: " + str(len(response[3])))
        for user in response[3]:
            unknown = True
            
            for u in user_list:
                if (user['_id'] == u['_id']):
                    unknown = False
                    break
            if (unknown == True):
                user_list.append(user)
        #inserci'on de tweets nuevos a la lista de tweets
        tweet_list.extend(response[2])
        

        total = response[0]
        tweet_count += response[0]
        #tweet_list.extend(total[2])
        
        endTime = time.time()
        aux = endTime - starTime
        if (aux < TIMER):
            time.sleep(math.ceil(TIMER - aux))
            tiempo -= TIMER
        else:
            tiempo -= aux

        if (tiempo <= 0):
            #aqui se imprime en el log los hashtags que se han escrito y c'uantos valores se han obtenido de estos
            tiempo = TIMER_LOG
            # datetime object containing current date and time
            now = datetime.datetime.now()
            # dd/mm/YY H:M:S
            dt_string = now.strftime("%H:%M:%S")
            hashtag_last_log = hashtag
            log.write(hashtag_ini_log + ' ' + start_log_timer + ' ' + str(tweet_count) + ' ' + hashtag + ' ' + dt_string + '\n')
            start_log_timer = dt_string
            tweet_count = 0
            hashtag_ini_log = hashtag_last_log
            
    else:
        startTime = 0
        endTime = 0
        while ((number - total) > 0):
            #Caso base
            if ((number - total) >= PETITIONS_LIMIT):
                startTime = time.time()
                response =  PetitionsLessEqual100(hashtag, start, end, PETITIONS_LIMIT, bearer, response[1])
                # insercion de usuarios nuevos y de tweets nuevos de usuarios ya insertados   
                for user in response[3]:
                    unknown = True
                    for u in user_list:
                        if (user['_id'] == u['_id']):
                            unknown = False
                    if (unknown == True):
                        #print("insercion unknown")
                        user_list.append(user)
                #inserci'on de tweets nuevos a la lista de tweets
                tweet_list.extend(response[2])
                endTime = time.time()
            #Fin de caso
            elif ((number - total) > 0) and ((number - total) < PETITIONS_LIMIT):
                startTime = time.time()
                if ((number - total) < 10):
                    response =  PetitionsLessEqual100(hashtag, start, end, 10, bearer, response[1])
                else:
                    response =  PetitionsLessEqual100(hashtag, start, end, (number - total), bearer, response[1])
                # insercion de usuarios nuevos y de tweets nuevos de usuarios ya insertados
                for user in response[3]:
                    unknown = True
                    for u in user_list:
                        if (user['_id'] == u['_id']):
                            unknown = False
                    if (unknown == True):
                        #print("insercion unknown")
                        user_list.append(user)
                #inserci'on de tweets nuevos a la lista de tweets
                tweet_list.extend(response[2])
                endTime = time.time()

            aux = endTime - startTime
            if (aux < TIMER):
                time.sleep(math.ceil(TIMER - aux))
                tiempo -= TIMER
            else:
                tiempo -= aux
                
            total += response[0]
            tweet_count += response[0]

            if (tiempo <= 0):
                #aqui se imprime en el log los hashtags que se han escrito y c'uantos valores se han obtenido de estos
                tiempo = TIMER_LOG
                # datetime object containing current date and time
                now = datetime.datetime.now()
                # dd/mm/YY H:M:S
                dt_string = now.strftime("%H:%M:%S")
                hashtag_last_log = hashtag
                log.write(hashtag_ini_log + ' ' + start_log_timer + ' ' + str(tweet_count) + ' ' + hashtag_last_log + ' ' + dt_string + '\n')
                start_log_timer = dt_string
                tweet_count = 0
                hashtag_ini_log = hashtag_last_log
            
            # caso en el que la respuesta no encuentra tweets o no hay mas tweets
            if (response[0] == 0) or ((response[0] > 0) and response[1] == ''):
                break
    
    print ("Tweets totales: " + str(total))
    print("Fin Hashtag: " + str(hashtag) + "\\\\\\\\\\\\ \n\n")

    #Aqui se inserta en la base de datos
    MongoExecute(tweet_list, user_list)

    return (total, hashtag, tweet_list, user_list)


def MongoExecute(tweet_list, user_list):
    global myclient
    global myusers
    global mytweets
    global mydb

    try:
        print(str(mytweets.insert_many(tweet_list, ordered=False)))
        print(str(myusers.insert_many(user_list, ordered=False)))
    except Exception as e:
        print(e)
        pass
        #cuando sepa hacerlo aquí haría una búsqueda con comprobación de que el usuario está en la lista y si está vacía completar los valores que le faltan.






    
    


## Main code

In [4]:

fileObj = Path("./log" + d1 + ".txt")
if (fileObj.is_file()):
    os.remove("./log" + d1 + ".txt")

fileObj = Path("./errorlog" + d1 + ".txt")
if (fileObj.is_file()):
    os.remove("./errorlog" + d1 + ".txt")

log = open("./log" + d1 + ".txt", "a")
log.write("Primer log\n")
#Crear ficheros de log
errorlog = open("./errorlog" + d1 + ".txt", "a")


f = open("./accountV2.txt", "r")
Bearer = f.readline().rstrip('\n')
f.close()

#####Uso de Hashtag
f =  open("./hashtags.txt", "r")
hashtagsList = []
hashtag = None
resultado = (0, '', [], [])

while True:
    # read line
    try:
        hashtag = f.readline()
        if (hashtag == "") :
            break
    except:
        # datetime object containing current date and time
        now = datetime.datetime.now()
        # dd/mm/YY H:M:S
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        errorlog.write("Fallo al leer el fichero hashtags.txt " + dt_string + "\n")

    hashtag = hashtag.rstrip('\n')
    hashtag = hashtag.replace('#', '')
    initDate = f.readline().rstrip('\n')
    lastDate = f.readline().rstrip('\n')
    hashtag = (hashtag, initDate, lastDate)
    hashtagsList.append(hashtag)
    # check if line is not empty
    
f.close()

for h in hashtagsList:
    print(h)

if (len(hashtagsList) != 0):
    hashtag_ini_log = hashtagsList[0]
    # dd/mm/YY H:M:S
    now = datetime.datetime.now()
    dt_string = now.strftime("%H:%M:%S")
    start_log_timer = dt_string
    
else:
    # datetime object containing current date and time
    now = datetime.datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    errorlog.write("Error: lista de hashtags vacía" + " dt_string\n")

myclient = pymongo.MongoClient('localhost', 27017)

mydb = myclient["mydatabase"]
myusers = mydb["users"]
mytweets = mydb["tweets"]

print(str(mydb.list_collection_names()))



for l in hashtagsList:
    
    resultado = TweetList(l[0], l[1], l[2], 100, Bearer)
    
    print("Tweet list")
    for tweet in resultado[2]:
        print(" id: " + str(tweet['_id']))
        print(" author_id: " + tweet['author_id'])
        print(" created_at: " + tweet['created_at'])
        print(" referenced_tweets: " + str(tweet['referenced_tweets']) + '\n')

    print("User list")
    for user in resultado[3]:
        print(" user_id: " + str(user['_id']))
        print(" description: " + user['description'])
        print(" created_at: " + user['created_at'])
        print(" verified: " + str(user['verified']))
        print(" followers_count: " + str(user['followers_count']))
        print(" following_count: " + str(user['following_count']) + '\n')
    
if (tiempo > 0):
    # datetime object containing current date and time
    now = datetime.datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%H:%M:%S")
    hashtag_last_log = hashtagsList[len(hashtagsList) - 1]
    log.write(str(hashtag_ini_log) + ' ' + start_log_timer + ' ' + str(tweet_count) + ' ' + str(hashtag_last_log) + ' ' + dt_string+ '\n')

log.close()
errorlog.close()


('DemocraciaOFascismo4M', '01-02-2021', '25-04-2021')
[]
Inicio Hashtag: DemocraciaOFascismo4M\\\\\\ 

N'umero de peticiones enviadas: 100
N'umero de peticiones recibidas: 99
N'umero de peticiones enviadas: 10
N'umero de peticiones recibidas: 10
Tweets totales: 109
Fin Hashtag: DemocraciaOFascismo4M\\\\\\ 


Tweet list
 id: 000001386288875023310854
 author_id: 1203613905882365952
 created_at: 2021-04-25T11:59:58.000Z
 referenced_tweets: []

 id: 000001386288874280919040
 author_id: 2694359793
 created_at: 2021-04-25T11:59:58.000Z
 referenced_tweets: [{'type': 'retweeted', 'id': '1386272541979910145', 'author_id': '372812630'}]

 id: 000001386288866865278976
 author_id: 308718847
 created_at: 2021-04-25T11:59:56.000Z
 referenced_tweets: [{'type': 'retweeted', 'id': '1386272541979910145', 'author_id': '372812630'}]

 id: 000001386288865896325125
 author_id: 851002164
 created_at: 2021-04-25T11:59:56.000Z
 referenced_tweets: [{'type': 'retweeted', 'id': '1386272541979910145', 'author_id':